In [5]:
# install crf and nltk in python if not installed
%pip install python-crfsuite
%pip install nltk
%pip install bs4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.4 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=e62651ccbd48d9a895bd490c2814b4bdc45881336c643a64733ca289cf20ba6c
  Stored in directory: /Users/shreyasv/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [7]:
from bs4 import BeautifulSoup as bs
from bs4.element import Tag
import codecs

# Read data file and parse the XML
with codecs.open("reuters.xml", "r", "utf-8") as infile:
    soup = bs(infile, "lxml")

docs = []
for elem in soup.find_all("document"):
    texts = []

    # Loop through each child of the element under "textwithnamedentities"
    for c in elem.find("textwithnamedentities").children:
        if type(c) == Tag:
            if c.name == "namedentityintext":
                label = "N"  # part of a named entity
            else:
                label = "I"  # irrelevant word
            for w in c.text.split(" "):
                if len(w) > 0:
                    texts.append((w, label))
    docs.append(texts)
    #Prepare labels as "N" representing part of a named entity and "I" for irrelevant word

docs[0]

/Users/shreyasv/Desktop/NLP/.venv/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


[('Paxar', 'N'),
 ('Corp', 'N'),
 ('said', 'I'),
 ('it', 'I'),
 ('has', 'I'),
 ('acquired', 'I'),
 ('Thermo-Print', 'N'),
 ('GmbH', 'N'),
 ('of', 'I'),
 ('Lohn', 'N'),
 (',', 'I'),
 ('West', 'N'),
 ('Germany', 'N'),
 (',', 'I'),
 ('a', 'I'),
 ('distributor', 'I'),
 ('of', 'I'),
 ('Paxar', 'N'),
 ('products,', 'I'),
 ('for', 'I'),
 ('undisclosed', 'I'),
 ('terms.', 'I')]

In [10]:
import nltk
nltk.download('averaged_perceptron_tagger', download_dir="/Users/shreyasv/Desktop/NLP/Lab 7")
nltk.data.path.append("/Users/shreyasv/Desktop/NLP/Lab 7")
data = []
for i, doc in enumerate(docs):

    # Obtain the list of tokens in the document
    tokens = [t for t, label in doc]

    # Perform POS tagging
    tagged = nltk.pos_tag(tokens)

    # Take the word, POS tag, and its label
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])
data[0]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/shreyasv/Desktop/NLP/Lab 7...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('Paxar', 'NNP', 'N'),
 ('Corp', 'NNP', 'N'),
 ('said', 'VBD', 'I'),
 ('it', 'PRP', 'I'),
 ('has', 'VBZ', 'I'),
 ('acquired', 'VBN', 'I'),
 ('Thermo-Print', 'NNP', 'N'),
 ('GmbH', 'NNP', 'N'),
 ('of', 'IN', 'I'),
 ('Lohn', 'NNP', 'N'),
 (',', ',', 'I'),
 ('West', 'NNP', 'N'),
 ('Germany', 'NNP', 'N'),
 (',', ',', 'I'),
 ('a', 'DT', 'I'),
 ('distributor', 'NN', 'I'),
 ('of', 'IN', 'I'),
 ('Paxar', 'NNP', 'N'),
 ('products,', 'NN', 'I'),
 ('for', 'IN', 'I'),
 ('undisclosed', 'JJ', 'I'),
 ('terms.', 'NN', 'I')]

In [11]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features


In [12]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [13]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 13357
Seconds required: 0.031

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 5689.585762
Feature norm: 1.000000
Error norm: 6301.928097
Active features: 12885
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 0.005

***** Iteration #2 *****
Loss: 4529.836698
Feature norm: 0.839162
Error norm: 5637.019723
Active features: 12999
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.003

***** Iteration #3 *****
Loss: 3943.467521
Feature norm: 0.823301
Error norm: 12063.056128
Active features: 8873
Line search trials: 2
Line search step: 0.500000
Seconds required for this 

In [14]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))


southmark (N)
corp (N)
said (I)
it (I)
acquired (I)
28 (I)
long-term (I)
care (I)
facilities (I)
containing (I)
for (I)
approximately (I)
70 (I)
mln (I)
dlrs (I)
in (I)
cash. (I)
it (I)
said (I)
the (I)
facilities, (I)
which (I)
contain (I)
approximately (I)
2,500 (I)
beds (I)
in (I)
seven (I)
western (I)
states, (I)
were (I)
bought (I)
from (I)
don (N)
bybee (N)
and (I)
associates (N)
, (I)
of (I)
salem (N)
, (I)
ore (N)
. (I)
the (I)
acquistion (I)
brings (I)
to (I)
57 (I)
health (I)
care (I)
facilities (I)
acquired (I)
in (I)
the (I)
last (I)
three (I)
months, (I)
the (I)
company (I)
said. (I)


In [15]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"N": 1, "I": 0}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["I", "N"]))


              precision    recall  f1-score   support

           I       0.98      0.98      0.98      3371
           N       0.85      0.83      0.84       346

    accuracy                           0.97      3717
   macro avg       0.92      0.91      0.91      3717
weighted avg       0.97      0.97      0.97      3717



In [22]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.5,

    # coefficient for L2 penalty
    'c2': 0.001,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf_1.model')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 13357
Seconds required: 0.032

L-BFGS optimization
c1: 0.500000
c2: 0.001000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 5692.723459
Feature norm: 1.000000
Error norm: 6303.712273
Active features: 5628
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 0.005

***** Iteration #2 *****
Loss: 4532.394729
Feature norm: 0.839032
Error norm: 5637.379431
Active features: 7194
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.002

***** Iteration #3 *****
Loss: 3948.503410
Feature norm: 0.822219
Error norm: 12066.728235
Active features: 4118
Line search trials: 2
Line search step: 0.500000
Seconds required for this it

In [23]:
tagger = pycrfsuite.Tagger()
tagger.open('crf_1.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))


southmark (N)
corp (N)
said (I)
it (I)
acquired (I)
28 (I)
long-term (I)
care (I)
facilities (I)
containing (I)
for (I)
approximately (I)
70 (I)
mln (I)
dlrs (I)
in (I)
cash. (I)
it (I)
said (I)
the (I)
facilities, (I)
which (I)
contain (I)
approximately (I)
2,500 (I)
beds (I)
in (I)
seven (I)
western (I)
states, (I)
were (I)
bought (I)
from (I)
don (N)
bybee (N)
and (N)
associates (N)
, (I)
of (I)
salem (N)
, (I)
ore (N)
. (I)
the (I)
acquistion (I)
brings (I)
to (I)
57 (I)
health (I)
care (I)
facilities (I)
acquired (I)
in (I)
the (I)
last (I)
three (I)
months, (I)
the (I)
company (I)
said. (I)


In [24]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"N": 1, "I": 0}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["I", "N"]))


              precision    recall  f1-score   support

           I       0.98      0.99      0.98      3371
           N       0.86      0.82      0.84       346

    accuracy                           0.97      3717
   macro avg       0.92      0.90      0.91      3717
weighted avg       0.97      0.97      0.97      3717



In [25]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.5,

    # coefficient for L2 penalty
    'c2': 0.1,  

    # maximum number of iterations
    'max_iterations': 400,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf_2.model')


Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 13357
Seconds required: 0.031

L-BFGS optimization
c1: 0.500000
c2: 0.100000
num_memories: 6
max_iterations: 400
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 5692.822459
Feature norm: 1.000000
Error norm: 6303.871028
Active features: 5628
Line search trials: 1
Line search step: 0.000044
Seconds required for this iteration: 0.005

***** Iteration #2 *****
Loss: 4532.442303
Feature norm: 0.839026
Error norm: 5637.464759
Active features: 7194
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.002

***** Iteration #3 *****
Loss: 3948.609566
Feature norm: 0.822154
Error norm: 12066.665817
Active features: 4118
Line search trials: 2
Line search step: 0.500000
Seconds required for this it

In [26]:
tagger = pycrfsuite.Tagger()
tagger.open('crf_2.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))


southmark (N)
corp (N)
said (I)
it (I)
acquired (I)
28 (I)
long-term (I)
care (I)
facilities (I)
containing (I)
for (I)
approximately (I)
70 (I)
mln (I)
dlrs (I)
in (I)
cash. (I)
it (I)
said (I)
the (I)
facilities, (I)
which (I)
contain (I)
approximately (I)
2,500 (I)
beds (I)
in (I)
seven (I)
western (I)
states, (I)
were (I)
bought (I)
from (I)
don (N)
bybee (N)
and (N)
associates (N)
, (I)
of (I)
salem (N)
, (I)
ore (N)
. (I)
the (I)
acquistion (I)
brings (I)
to (I)
57 (I)
health (I)
care (I)
facilities (I)
acquired (I)
in (I)
the (I)
last (I)
three (I)
months, (I)
the (I)
company (I)
said. (I)


In [27]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"N": 1, "I": 0}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["I", "N"]))


              precision    recall  f1-score   support

           I       0.98      0.99      0.98      3371
           N       0.85      0.83      0.84       346

    accuracy                           0.97      3717
   macro avg       0.92      0.91      0.91      3717
weighted avg       0.97      0.97      0.97      3717

